# Regresión

Autores:
Ivan Toribio
Héctor Sancho
Jordi Hamberg

# Enunciado

En esta práctica, se os proporcionará un conjunto de datos con características de automóviles de segunda mano en distintos países. El objetivo es preprocesar los datos y construir un modelo capaz de predecir los precios de coches que nunca haya visto.

Además, se llevará a cabo una **competición** en kaggle, podéis inscribiros en este siguiente [enlace](https://www.kaggle.com/t/95194d38b1f9447fa9e3b51b8e9d575f)




La estructura del proyecto debe ser la siguiente:

- Carga y limpieza de los datos
- Visualización
- Preprocesamiento
- Pruebas de modelos (razonar sobre los resultados)
- Conclusiones
- Creación de submission



Se valorarán:

- Número y variedad de experimentos con sus conclusiones.
- Calidad del preprocesamiento (y su razonamiento).
- Capacidad de generalización del modelo final.
- Trabajo en equipo (en grupos de más de 1 integrante, claro).
- Creatividad en la solución.
- Claridad del .ipynb presentado en Moodle

# Carga y limpieza de los datos


Aunque se permite la carga de los datos de forma local, se recomienda emplear las siguientes URLs de gist por comodidad:

In [53]:
import numpy as np

In [54]:
train_url = "https://gist.githubusercontent.com/w-dan/9c2f9a30d0517c58a08d5a7ef42ef2dd/raw/ee85f61926c9aeae5863369a13bb67e182a702ba/training.csv"
test_url = "https://gist.githubusercontent.com/w-dan/9c2f9a30d0517c58a08d5a7ef42ef2dd/raw/ee85f61926c9aeae5863369a13bb67e182a702ba/test.csv"

In [55]:
import pandas as pd

train_df = pd.read_csv(train_url)
test_df = pd.read_csv(test_url)

In [56]:
import matplotlib.pyplot as plt

# Visualización

In [57]:
train_df

,id,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,1,Chevrolet Spark 1.0 LT,2011,130000,68519,Petrol,Dealer,Manual,First Owner,16.2 kmpl,995 CC,62.1 bhp,90.3Nm@ 4200rpm,5.0
1,2,Hyundai Verna 1.6 SX CRDi (O),2017,1050000,70000,Diesel,Dealer,Manual,First Owner,22.32 kmpl,1582 CC,126.32 bhp,259.8Nm@ 1900-2750rpm,5.0
2,3,Mahindra Scorpio S10 7 Seater,2016,1100000,60000,Diesel,Individual,Manual,First Owner,15.4 kmpl,2179 CC,120 bhp,280Nm@ 1800-2800rpm,7.0
3,4,Hyundai EON 1.0 Kappa Magna Plus Optional,2018,350000,10000,Petrol,Individual,Manual,Second Owner,20.3 kmpl,998 CC,68.05 bhp,94.14Nm@ 3500rpm,5.0
4,5,Maruti Swift Dzire VDI,2014,490000,50000,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6904,6905,Hyundai i20 2015-2017 Asta Option 1.2,2016,650000,29000,Petrol,Individual,Manual,First Owner,18.6 kmpl,1197 CC,81.83 bhp,114.7Nm@ 4000rpm,5.0
6905,6906,Hyundai EON Sportz,2017,300000,10000,Petrol,Individual,Manual,First Owner,21.1 kmpl,814 CC,55.2 bhp,74.5Nm@ 4000rpm,5.0
6906,6907,Maruti Swift VDI Optional,2017,693000,58559,Diesel,Dealer,Manual,First Owner,25.2 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
6907,6908,Maruti Baleno Alpha 1.2,2017,580000,20000,Petrol,Individual,Manual,First Owner,21.4 kmpl,1197 CC,83.1 bhp,115Nm@ 4000rpm,5.0


# Preprocesamiento

In [58]:
processed_df = pd.DataFrame()

In [59]:
def crear_pol(columna : pd.Series, column_name : str):

    DEGREE = 1

    if DEGREE == 1:
        return pd.DataFrame({column_name + "_1": columna})

    columna = columna.replace(np.nan, 0)

    matriz_valores = columna.values.reshape(-1, 1)

    from sklearn.preprocessing import PolynomialFeatures
    pol = PolynomialFeatures(DEGREE)

    matriz = pol.fit_transform(matriz_valores)

    df_temp = pd.DataFrame()

    for i, columna in enumerate(matriz.transpose()):
        df_temp[f"{column_name}_{i}"] = columna

    df_temp = df_temp.replace(0, np.nan)

    return df_temp

In [60]:
#nombre
processed_df["name"] = train_df["name"]

In [61]:
#precio
processed_df["selling_price"] = train_df["selling_price"]

In [62]:
#year
processed_df = pd.concat([processed_df, crear_pol(train_df["year"], "year")], axis=1)

In [63]:
#km_driven
processed_df = pd.concat([processed_df, crear_pol(train_df["km_driven"], "km_driven")], axis=1)

In [64]:
def splitter(x):
    if type(x) == str:
        return float(x.split(" ")[0])
    return x

In [65]:
import re

def splitter_re(x):
    if type(x) == str:
        try:
            return re.findall(r"[0-9.]+", x)[0]
        except Exception:
            return np.nan
    else:
        return x

In [66]:
#mileage

serie = train_df["mileage"]
serie = serie.apply(splitter)

processed_df = pd.concat([processed_df, crear_pol(serie, "mileage")], axis=1)



In [67]:
#engine


serie = train_df["engine"]
serie = serie.apply(splitter)

processed_df = pd.concat([processed_df, crear_pol(serie, "engine")], axis=1)

In [68]:
#max_power

serie = train_df["max_power"]
serie = serie.apply(splitter_re)

processed_df = pd.concat([processed_df, crear_pol(serie, "max_power")], axis=1)

In [69]:
#seats

serie = train_df["seats"]

processed_df = pd.concat([processed_df, crear_pol(serie, "seats")], axis=1)

In [70]:
categoricas = ["fuel", "seller_type", "transmission", "owner"]

diccionarios = {}

In [71]:
df_aux = pd.DataFrame({

    "selling_price":train_df["selling_price"],
    "brand": train_df["name"].map(lambda x: x.split()[0])

})

pesos = df_aux.groupby("brand").mean().reset_index()
pesos = pesos.rename(columns={"selling_price": "peso"})

df_aux.drop(columns=["selling_price"], inplace=True)

diccionario = {categorica : peso for categorica, peso in pesos.values}

diccionarios["brand"] = diccionario

df_aux = df_aux.map(lambda x: diccionario[x])

processed_df = pd.concat([processed_df, crear_pol(df_aux["brand"], "brand")], axis=1)

In [72]:
for categorica in categoricas:

    df_aux = train_df.loc[:,[categorica, "selling_price"]]

    pesos = df_aux.groupby(categorica).mean().reset_index()
    pesos = pesos.rename(columns={"selling_price": "peso"})

    df_aux.drop(columns=["selling_price"], inplace=True)

    diccionario = {categorica : peso for categorica, peso in pesos.values}

    diccionarios[categorica] = diccionario

    df_aux = df_aux.map(lambda x: diccionario[x])

    processed_df = pd.concat([processed_df, crear_pol(df_aux[categorica], categorica)], axis=1)

# Pruebas de modelos

In [73]:
processed_df = processed_df.dropna()

from sklearn.model_selection import train_test_split

train, test = train_test_split(processed_df, test_size=0.2, random_state=43)


X_train = train.drop(["name", "selling_price"], axis=1)
y_train = train["selling_price"]

X_test = test.drop(["name", "selling_price"], axis=1)
y_test = test["selling_price"]

from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor()

from sklearn.ensemble import HistGradientBoostingRegressor
reg = HistGradientBoostingRegressor()

reg.fit(X_train, y_train)



HistGradientBoostingRegressor()

In [74]:
#metricas

from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import explained_variance_score as evs


print (mae(y_test, reg.predict(X_test))*0.011)
print (mae(y_test, reg.predict(X_test)))
print (evs(y_test, reg.predict(X_test)))

769.8641451216802
69987.64955651638
0.9713850234694744


In [81]:
df_resultados = test.loc[:, ['name', 'selling_price', 'year_1', 'km_driven_1', 'mileage_1',
       'engine_1', 'max_power_1', 'seats_1', 'brand_1', 'fuel_1',
       'seller_type_1', 'transmission_1', 'owner_1']]

df_resultados["precio"] = df_resultados["selling_price"] * 0.011
df_resultados["prediccion"] = reg.predict(X_test) * 0.011
df_resultados["diferencia"] = (df_resultados["prediccion"] - df_resultados["precio"])
df_resultados["rel_diferencia"] = (df_resultados["diferencia"] / df_resultados["precio"]) * 100

df_resultados

,name,selling_price,year_1,km_driven_1,mileage_1,engine_1,max_power_1,seats_1,brand_1,fuel_1,seller_type_1,transmission_1,owner_1,precio,prediccion,diferencia,rel_diferencia
6354,Tata Nano Cx,45000,2011,10000,26.00,624.0,35,4.0,360128.834395,459605.076059,4.920603e+05,453091.397115,278241.152401,495.0,1060.019315,565.019315,114.145316
6697,Tata Nano CX,95000,2013,30000,25.40,624.0,37.48,4.0,360128.834395,459605.076059,4.920603e+05,453091.397115,771708.327317,1045.0,1333.053841,288.053841,27.564961
5521,Tata Nexon 1.2 Revotron XM,537000,2020,5000,17.00,1198.0,108.5,5.0,360128.834395,459605.076059,4.920603e+05,453091.397115,771708.327317,5907.0,9272.154068,3365.154068,56.968919
5127,Maruti Alto K10 VXI,370000,2017,30000,23.95,998.0,67.05,5.0,405203.436681,459605.076059,4.920603e+05,453091.397115,771708.327317,4070.0,3973.157092,-96.842908,-2.379433
3641,Maruti Eeco 7 Seater Standard BSIV,370000,2016,58000,15.37,1196.0,73,7.0,405203.436681,459605.076059,4.920603e+05,453091.397115,385728.259053,4070.0,3641.247873,-428.752127,-10.534450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2511,Fiat Linea Emotion (Diesel),265000,2010,61000,20.40,1248.0,91.7,5.0,288717.897436,778718.659924,1.415780e+06,453091.397115,385728.259053,2915.0,2180.768189,-734.231811,-25.188055
2229,Maruti Swift Dzire VDi,185000,2009,138000,19.30,1248.0,73.9,5.0,405203.436681,778718.659924,4.920603e+05,453091.397115,385728.259053,2035.0,2476.591838,441.591838,21.699845
4606,Chevrolet Beat Diesel LT,185000,2014,90000,25.44,936.0,56.3,5.0,269997.621891,778718.659924,4.920603e+05,453091.397115,385728.259053,2035.0,2106.139672,71.139672,3.495807
5809,Maruti Alto LXI,105000,2004,96000,19.70,796.0,46.3,5.0,405203.436681,459605.076059,4.920603e+05,453091.397115,278241.152401,1155.0,902.175598,-252.824402,-21.889559


In [76]:
processed_df

,name,selling_price,year_1,km_driven_1,mileage_1,engine_1,max_power_1,seats_1,brand_1,fuel_1,seller_type_1,transmission_1,owner_1
0,Chevrolet Spark 1.0 LT,130000,2011,68519,16.20,995.0,62.1,5.0,269997.621891,459605.076059,1.415780e+06,453091.397115,771708.327317
1,Hyundai Verna 1.6 SX CRDi (O),1050000,2017,70000,22.32,1582.0,126.32,5.0,454804.856073,778718.659924,1.415780e+06,453091.397115,771708.327317
2,Mahindra Scorpio S10 7 Seater,1100000,2016,60000,15.40,2179.0,120,7.0,618376.889231,778718.659924,4.920603e+05,453091.397115,771708.327317
3,Hyundai EON 1.0 Kappa Magna Plus Optional,350000,2018,10000,20.30,998.0,68.05,5.0,454804.856073,459605.076059,4.920603e+05,453091.397115,385728.259053
4,Maruti Swift Dzire VDI,490000,2014,50000,23.40,1248.0,74,5.0,405203.436681,778718.659924,4.920603e+05,453091.397115,771708.327317
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6904,Hyundai i20 2015-2017 Asta Option 1.2,650000,2016,29000,18.60,1197.0,81.83,5.0,454804.856073,459605.076059,4.920603e+05,453091.397115,771708.327317
6905,Hyundai EON Sportz,300000,2017,10000,21.10,814.0,55.2,5.0,454804.856073,459605.076059,4.920603e+05,453091.397115,771708.327317
6906,Maruti Swift VDI Optional,693000,2017,58559,25.20,1248.0,74,5.0,405203.436681,778718.659924,1.415780e+06,453091.397115,771708.327317
6907,Maruti Baleno Alpha 1.2,580000,2017,20000,21.40,1197.0,83.1,5.0,405203.436681,459605.076059,4.920603e+05,453091.397115,771708.327317


# Conclusiones

# Creación de *submission* para Kaggle

In [77]:
#procesado dataset

processed_sub_df = pd.DataFrame()

#nombre
processed_sub_df["name"] = test_df["name"]

#year
processed_sub_df = pd.concat([processed_sub_df, crear_pol(test_df["year"], "year")], axis=1)

#km_driven
processed_sub_df = pd.concat([processed_sub_df, crear_pol(test_df["km_driven"], "km_driven")], axis=1)

#mileage

serie = test_df["mileage"]
serie = serie.apply(splitter)

processed_sub_df = pd.concat([processed_sub_df, crear_pol(serie, "mileage")], axis=1)

#engine


serie = test_df["engine"]
serie = serie.apply(splitter)

processed_sub_df = pd.concat([processed_sub_df, crear_pol(serie, "engine")], axis=1)

#max_power

serie = test_df["max_power"]
serie = serie.apply(splitter_re)

processed_sub_df = pd.concat([processed_sub_df, crear_pol(serie, "max_power")], axis=1)

#seats

serie = test_df["seats"]

processed_sub_df = pd.concat([processed_sub_df, crear_pol(serie, "seats")], axis=1)

#Categoricas

serie_marcas = test_df["name"].apply(lambda x: x.split(" ")[0])
serie_marcas = serie_marcas.map(lambda x: diccionarios["brand"][x])
processed_sub_df = pd.concat([processed_sub_df, crear_pol(serie_marcas, "brand")], axis=1)



categoricas = ["fuel", "seller_type", "transmission", "owner"]

for categorica in categoricas:
    serie = test_df[categorica].map(lambda x: diccionarios[categorica][x])
    processed_sub_df = pd.concat([processed_sub_df, crear_pol(serie, categorica)], axis=1)

    



In [80]:
X_sub = processed_sub_df.drop("name", axis=1)
prediccion = reg.predict(X_sub)


prediccion

array([ 135861.19644493, 1000594.62845934,  946972.61129694,
        388201.73890816,  482583.39564642,  222846.15136295,
        170241.92283674,  260391.31392532,  518185.83763761,
        180171.85085607, 1095318.979245  ,  411184.60672749,
        417752.39883901,  499450.71852088, 5151809.12668592,
        227293.6479287 ,  585722.99029804,  323938.45945319,
        513704.99213515,  447129.57601398,  503643.25760551,
        411243.55397125,  129448.7236152 ,  951604.86073525,
        556330.00891964,  793832.06278626,  186917.09676671,
        632748.42974146,  166383.4476614 ,  242623.84792303,
        678958.96430356,  505143.55946934, 5151809.12668592,
        738324.92618766,  600222.65900451, 2687085.58401625,
        229580.26840884,  146193.58129468,  857411.68055568,
        180411.00843462,  224767.54688647,  610673.93488418,
        259206.66445332,  477581.53890403,  647445.41336564,
        130300.72857809,  240589.77215054,  333365.13293183,
        179831.87638044,

In [79]:
from csv import writer

with open("resultados.csv", "w") as file:
    escritor = writer(file, lineterminator="\n")

    escritor.writerow(["id", "TARGET"])

    for i, dato in enumerate(prediccion):
        escritor.writerow([i + 1,dato])